In [6]:
import numpy as np
import os
import glob
import math
import shutil
import sys
from subprocess import Popen
mode = 'PAR'
main_dir = os.getcwd()
source_EoS_1 = os.path.join(main_dir,'EoS_1')
EoS_others_path = os.path.join(main_dir,'EoS_others')

##########################################################################################

# functions

def clear_inputs():
    print('!!! Code will clear parameters files in all input/parameters_dir')
    if input('enter y to conitnue, anything else to quit: ') == 'y':
        eos1_para_dir = os.path.join(source_EoS_1,'input/parameters_dir')
        for f in os.listdir(eos1_para_dir):
            os.remove(os.path.join(eos1_para_dir, f))
        for d in os.listdir(EoS_others_path):
            curr_para_path_others = os.path.join(EoS_others_path,d,'input/parameters_dir')
            for f in os.listdir(curr_para_path_others):
                os.remove(os.path.join(curr_para_path_others, f))
        print('\nParameter files deleted\nProgram initiated')
        return
    else:
        sys.exit('program terminated')
    return
    
def asker_parameters(): # gets info from user to determine what ranges and values of the parameters to use for the parameters.dat files
    # returns a dictionary with {'parameter type':[str of values]}
    print('input min value incremental value max value in form mv iv xv')
    all_dict = {'T0':[], 'K':[], 'muB':[], 'ang':[], 'w':[], 'p':[]  }
    for i in all_dict:
        yn = input('\nEnter y or n if %s will be constant: ' % i)
        if yn == 'y':
            all_dict[i]=[str(float(input('Enter constant value for %s: ' % i)))]
        elif yn == 'n':
            mv = float(input('Enter min value for %s: ' % i))
            xv = float(input('Enter max value for %s: ' % i))
            iv = float(input('Enter increment value for %s: ' % i))
            fvals = np.arange(mv,xv+iv,iv)
            svals = [str(f) for f in fvals]
            all_dict[i] = svals
        else:
            print('\n\n!!! Invalid entry. Parameters cleared. Try again.')
            return 0
    print('------------------------------------------------')
    q = 1 #quantity of parameter files to be created
    for key in all_dict:
        q *= len(all_dict[key])
    print('Total number of parameter files to run is %i' % q)
    yn = input('Are all the above settings correct? (y or n): ')
    while yn != 'y' and yn != 'n':
        print('!!! Invalid entry')
        yn = input('Are the above settings correct? (y or n): ')
    if yn == 'y':
        return all_dict
    else:
        print('\n!!!Parameters cleared. Reenter new parameters.\n')
        print('////////////////////////////////////////\n')
        return 0

def parameters(): # returns a list of all parameters
    p_list = []
    c = 0
    for key in all_dict:
        if c == 0:
            for val in all_dict[key]:
                p_list.append(val)
            c = 1
            continue
        else:
            tmp_list = []
            for d_val_elm in all_dict[key]:
                for p_list_elm in p_list:
                    tmp_list.append(p_list_elm+' '+d_val_elm)
            p_list = tmp_list
    tmp_list = []
    for elm in p_list:
        tmp_list.append(mode+' '+elm)
    p_list = tmp_list
    return p_list

def parameter_assigner(p_list,EoS_list):
    # make list of optimal quantities for each eos instance
    n_p, n_e = len(p_list),len(EoS_list)
    r_up = int(math.ceil(n_p/n_e))
    r_dwn = int(math.floor(n_p/n_e))
    s_n = n_e-1
    s_f = n_p-(s_n*r_up)
    nofrdwn = r_up - s_f # delta
    nofrup = n_e - nofrdwn
    r_up_list = [r_up for x in range(0,nofrup)]
    r_dwn_list = [r_dwn for x in range(0,nofrdwn)]
    r_list_final = r_up_list+r_dwn_list
    
    # write files to repective eos parameters_dir
    eos_paths_list = []
    bot,top,i,eos_num = 0,0,0,1 #bot number of range, top number of range, index of list, eos number
    for n in r_list_final : # for each quantity for each eos, n is the amount of parameter files per EoS instance
        if eos_num == 1:
            curr_eos_path = source_EoS_1
        else:
            curr_eos_path = os.path.join(EoS_others_path,'EoS_'+str(eos_num))
        para_dir = os.path.join(curr_eos_path,'input/parameters_dir')
        top += n
        for x in range(bot,top): # for each element in p_list
            input_inside = p_list[x]
            input_title = input_inside.replace(" ", "_")+'.dat'
            input_filename = os.path.join(para_dir, input_title)
            f = open(input_filename,'w')
            f.write(input_inside)
            f.close()
        bot += n
        eos_num += 1
        eos_paths_list.append(curr_eos_path)
    return eos_paths_list

def lister_EoS():
    tmp_EoS_list = os.listdir(EoS_others_path)
    tmp_EoS_list.append('EoS_1')
    sorted(tmp_EoS_list)
    len_tmp_list = len(tmp_EoS_list)
    return tmp_EoS_list, len_tmp_list

def asker_EoS(len_p_list): # asks for how many EoS instances desired, and then runs EoS_maker()
    EoS_list, len_EoS_list = lister_EoS()
    print('\n------------------------------------------------')
    print('There are %i EoS instances' % len_EoS_list)
    yi = input('To run with %i EoS instance(s) enter y, else enter an integer of how many instances are desired: ' % len_EoS_list)
    while yi != 'y':
        if yi.isdigit(): # checking for integers
            if int(yi) > len_p_list:
                print('!!! Requesting more EoS than there are parameter files to run.')
                print('!!! Will bring number of EoS instances to amount of paramter files. So one parameter file will be fun on each EoS instance.')
                EoS_maker(len_p_list, len_EoS_list)
            else:
                EoS_maker(int(yi), len_EoS_list)
            EoS_list, len_EoS_list = lister_EoS()
            return EoS_list, len_EoS_list
        else:
            print('!!! Invalid entry.')
            yi = input('To run with this many EoS instances enter y, else enter an integer: ')
            continue
    verify_state = verify(len_p_list,len_EoS_list)
    if verify_state:
        EoS_list, len_EoS_list = lister_EoS()
    else:
        pass
    return EoS_list, len_EoS_list

def verify(len_p_list,len_EoS_list): # verifies that no. of parameter files is not smaller than no. of EoS instances. If so it reduceds EoS instances to what is needed only.
    if len_p_list < len_EoS_list:
        print("!!! You've selected more EoS instances than required for the number of parameter files to be run.")
        print("!!! EoS instances will be reduced to %i instances." % len_p_list)
        EoS_maker(len_p_list,len_EoS_list)
        return True
    else:
        return        

def EoS_maker(yi, len_EoS_list): # args are ints (desired isntances, actual intances) . This creates, maintains, or deletes EoS instances
    if yi == len_EoS_list:
        print("!!! You've entered the same number of EoS instances that already exits.")
        return
    elif yi > len_EoS_list: # copies EoS_1 into EoS_others and renames chronologically
        for i in range(len_EoS_list+1,yi+1):
            EoS_dirname = 'EoS_'+str(i)
            destination = os.path.join(EoS_others_path,EoS_dirname)
            shutil.copytree(source_EoS_1, destination)
        return
    else:
        for i in range(len_EoS_list,yi,-1): # deletes directories to get desired no. of instances
            if i == 1: # in case someone puts 0, keeps EoS_1 regardless
                print("!!! You've entered smaller than 1 instances.") 
                print("All EoS in EoS_others were deleted, and EoS_1 was kept.")
                break
            else:
                EoS_dirname = 'EoS_'+str(i)
                destination = os.path.join(EoS_others_path,EoS_dirname)
                shutil.rmtree(destination)
    return

def execute_cmds():
    for path in eos_paths_list:
        os.chdir(path) # this is to point to the correct directory. runners dont work otherwise bc they call from the cwd, which needs to be changed beforehand
        runner_path = os.path.join(path,'EoS_runner.py')
        inargs = ["python",runner_path]
        Popen(inargs)

##########################################################################################

# main

clear_inputs() #clears inputs to begin
check = asker_parameters()
while check == 0: # repeats if invalid entry for asker_parameters()
    check = asker_parameters()
all_dict = check # final settings for parameters saved to all_dict
p_list = parameters() # makes the list of parameters strings
len_p_list = len(p_list)
EoS_list, len_EoS_list = asker_EoS(len_p_list) # gets number of EoS instances wanted and creates or deletes to achieve

print("\nCode will continue with %i EoS instances" % len_EoS_list)
final_check = input("\nEnter y to continue or anything else to terminate program: ")
if final_check == 'y':
    pass
else:
    sys.exit('program terminated')

eos_paths_list = parameter_assigner(p_list, EoS_list) # assigns and delegates parameter files and return a list of all EoS instances paths
execute_cmds() # executes runner scripts that run the eoses
os.chdir(main_dir) # brings back to main directory so process can be repeated. if not, python is pointing to the last eos directory called

!!! Code will clear parameters files in all input/parameters_dir
enter y to conitnue, anything else to quit: y

Parameter files deleted
Program initiated
input min value incremental value max value in form mv iv xv

Enter y or n if T0 will be constant: y
Enter constant value for T0: 155

Enter y or n if K will be constant: y
Enter constant value for K: -0.0149

Enter y or n if muB will be constant: n
Enter min value for muB: 350
Enter max value for muB: 550
Enter increment value for muB: 50

Enter y or n if ang will be constant: y
Enter constant value for ang: 90

Enter y or n if w will be constant: n
Enter min value for w: 1
Enter max value for w: 4
Enter increment value for w: 1

Enter y or n if p will be constant: y
Enter constant value for p: 1
------------------------------------------------
Total number of parameter files to run is 20
Are all the above settings correct? (y or n): y

------------------------------------------------
There are 6 EoS instances
To run with 6 EoS insta